In [ ]:
pip install python-crfsuite

In [ ]:
pip install scikit-learn

In [ ]:
pip install sklearn-crfsuite

In [ ]:
pip install nbimporter

In [ ]:
pip install ipynb

In [9]:
import sys
import os
import numpy as np
import pandas as pd
import json
import pickle
from zipfile import ZipFile
import sklearn
from sklearn.model_selection import train_test_split
import pycrfsuite
import scipy.stats
import sklearn_crfsuite

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\Map_Processing")

from analyze_notes import get_placement_features 

In [3]:
# Get maps dataframe from the pickle file
maps_df = pd.read_pickle("../Data_Gather_Filter_Download/downloaded_maps_df.pkl")

with open('../Note_Orientation/most_common_placements.pkl', 'rb') as f:
    most_common_placements = pickle.load(f)

In [4]:
#==================================== Dataset Settings ====================================#
# Note: We have to use a subset of all our songs as 15k maps would take days to train
total_data_size = 1000  # Number of maps to use in across all datasets
val_split = 0.15        # Percentage of data put into validation set
test_split = 0.15       # Percentage of data put into testing set
#==========================================================================================#

# Split our data into training and test/val which we will split again
train_data, val_test_data = train_test_split(maps_df[:total_data_size], test_size=val_split + test_split)

# Split the validation and testing data apart into their own respective sets
val_data, test_data = train_test_split(val_test_data, test_size=(test_split/(val_split + test_split)))

print("Number of training maps: {}. Numer of validation maps: {}. Number of testing maps: {}".format(len(train_data), len(val_data), len(test_data)))

Number of training maps: 700. Numer of validation maps: 150. Number of testing maps: 150


In [15]:
# Get training data into format that can be read by model
all_training_features = [] # Will be list of list of dictionaries
all_training_labels = []
for map_data in train_data.itertuples():
    file_path = map_data.file_path
    if file_path != "NOT_FOUND":
        with ZipFile("../Data_Gather_Filter_Download/{}".format(file_path)) as folder:
            try:
                # Open the dat file for the difficulty
                if map_data.difficulty == 'expert':
                    difficulty_dat = "Expert.dat"
                else:
                    difficulty_dat = "ExpertPlus.dat"
                with folder.open(difficulty_dat) as diff_dat:
                    dat_json = json.load(diff_dat)
                    # Get the features for an entire song (dictionary with time points key feature dictionary values)
                    features_at_time_points = get_placement_features(dat_json, most_common_placements)
                    map_labels = []
                    for time_point, features in features_at_time_points.items():
                        features.pop('placement', None)
                        map_labels.append(("{}".format(features['placement_index'])).encode('utf-8'))
                        # Needs to be encoded this way for the model
                        # features['placement_index'] = "{}".format(features['placement_index'])
                        # features['prev_placement'] = "{}".format(features['prev_placement'])
                        # features['next_placement'] = "{}".format(features['next_placement'])
                    all_training_features.append(list(features_at_time_points.values()))
                    all_training_labels.append(map_labels)
            except Exception as e:
                print(e, file_path)

In [29]:
# Declare our CRF model using SKLearn
crf_model = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100)

In [30]:
%%time
# Train our model
crf_model.fit(all_training_features, all_training_labels)

KeyboardInterrupt: 

In [28]:
print(json.dumps(all_training_features[1], indent=4))

281",
        "time_since_last_note": "1.0",
        "time_to_next_note": "1.0",
        "num_notes": "2",
        "colours": [
            "0",
            "1"
        ],
        "rows": [
            "0",
            "1"
        ],
        "cols": [
            "3",
            "3"
        ],
        "directions": [
            "3",
            "3"
        ],
        "placement_count": "0",
        "prev_placement": "10",
        "next_placement": "82"
    },
    {
        "placement_index": "82",
        "time_point": "348.3520812988281",
        "time_since_last_note": "1.0",
        "time_to_next_note": "1.0",
        "num_notes": "2",
        "colours": [
            "1",
            "0"
        ],
        "rows": [
            "0",
            "1"
        ],
        "cols": [
            "0",
            "0"
        ],
        "directions": [
            "2",
            "2"
        ],
        "placement_count": "0",
        "prev_placement": "81",
        "next_placement": "81"